In [1]:
import sys
sys.path.append('../libs')

In [2]:
import pandas as pd
import numpy as np

In [4]:
from model_evaluation_helpers import *
from visualization_helpers import *

In [5]:
# %load_ext autoreload
# %autoreload 2

### Load Data

In [6]:
# List of path/modelID tuples
path_to_models = [
    ("../dummy_data/embeds/baseline_embs.csv", "Baseline"),
    ("../dummy_data/embeds/model_embs.csv", "Model")
]

In [7]:
path_to_annotations = "../dummy_data/text/annotated_tweets.csv"

In [8]:
# Load annotated tweets
annotations = pd.read_csv(path_to_annotations)
annotations = annotations.rename(columns={'nodeID':'tweet_id'})

### Compute kNN metrics

##### Parameters

In [9]:
groupby_cols = ['model_id', 'N']
pca_norm = False # If True apply PCA to embeddings
n_neighbors = [1,3,5,7,9,11,13,15] # Number of K values to consider

##### Compute ViewpointNN

In [10]:
viewpoint_df = []
for n in n_neighbors:
    df = compute_nn_metrics(path_to_models, 
                            test_df=annotations, 
                            n=n, 
                            metric='viewpoint', 
                            pca_norm=False)
    viewpoint_df.append(df)
viewpoint_df = pd.concat(viewpoint_df, ignore_index=True)
viewpoint_df['prediction'] = viewpoint_df['actual_class'] == viewpoint_df['majority_class']

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 54.97it/s]


In [11]:
results1 = (
    viewpoint_df
    .groupby(groupby_cols)
    .apply(lambda x: x['prediction'].sum()/len(x)) # sum TP divide by num of instances
    .reset_index(name='viewpointNN')
)
results1.head(5).round(4)

,model_id,N,viewpointNN
0,Baseline,1,0.55
1,Baseline,3,0.55
2,Baseline,5,0.55
3,Baseline,7,0.55
4,Baseline,9,0.35


##### Compute TopicNN

In [12]:
topic_df = []
for n in n_neighbors:
    df = compute_nn_metrics(path_to_models,
                            test_df=annotations,
                            n=n,
                            metric='topic',
                            pca_norm=False)
    topic_df.append(df)
topic_df = pd.concat(topic_df, ignore_index=True)
topic_df['prediction'] = topic_df['majority_class']

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 31.19it/s]


In [13]:
results2 = (
    topic_df
    .groupby(groupby_cols)
    .apply(lambda x: x['prediction'].sum()/len(x)) # sum TP divide by num of instances
    .reset_index(name='topicNN')
)
results2.head(5).round(4)

,model_id,N,topicNN
0,Baseline,1,1.0
1,Baseline,3,1.0
2,Baseline,5,1.0
3,Baseline,7,1.0
4,Baseline,9,1.0


##### Merge Results

In [14]:
results = pd.merge(results1, results2, on=groupby_cols, how='inner')

### Visualizations

##### Scatter plot: ViewpointNN vs. TopicNN at K

In [ ]:
K = 5

# Dictionary of model_ids and corresponding label
label_map = {
    'Baseline':'Baseline',
    'Model':'Model'
}

# Dictionary of model_ids and corresponding color
color_map = {
    'Baseline':'black',
    'Model':'red',
}

# Parameters for plot
plot_params = {
    'x':'topicNN',
    'y':'viewpointNN',
    'hue_col':'model_id',
    'color_map':color_map,
    'label_map':label_map,
    'figsize':(5,5),
    'markersize':300,
    'horizontal_bound':None,
    'vertical_bound':None,
    'hue_order':None
}

In [ ]:
plot_df = results.copy()
plot_df = plot_df.loc[plot_df['N']==K].reset_index(drop=True)

In [ ]:
fig = plot_scatter(plot_df, params=plot_params)

##### Line plot: performance at varying K

In [ ]:
plot_params = {
    'x':'N',
    'y':'viewpointNN', # viewpointNN or topicNN
    'hue_col':'model_id',
    'color_map':color_map,
    'label_map':label_map,
    'figsize':(8,5),
    'hue_order':None
}

In [ ]:
fig = plot_line(results, params=plot_params)

##### UMAP: embeddings visualization

In [ ]:
# Load embeddings
embs_df = pd.read_csv(path_to_models[0][0], dtype={"tweet_id":str}) 

In [ ]:
# add topic and stance information to embeddings
embs_df = pd.merge(embs_df, annotations[['tweet_id', 'topic', 'stance']], on='tweet_id', how='left')

In [ ]:
plot_embeddings(embs_df)